# Create working dataset

To be able to illustrate all analysis steps using realistic genetic data, we have prepared a dataset using data from the HapMap Project.

This preparation consist on updating the datasets to rs, removing SNPs with minor allele frequency and adding fake case/control phenotypes. We have also purposely duplicated one individual to check that the algorithms detects it.

Working dataset is on build 36 purposely, in order to demonstrate how to change builds. This is done in Task1_preQC_template.ipynb.

**TODO comment and execute. Copy ref files from Zeus**

In [3]:
%env Path=/mnt/data/GWAS

env: Path=/mnt/data/GWAS


In [7]:
%%bash
head $Path/input/mydataset.b37.rs.bim
tail $Path/input/mydataset.b37.rs.bim
wc $Path/input/mydataset.b37.rs.fam

1	1:10177	0	10177	AC	A
1	1:10352	0	10352	TA	T
1	1:11008	0	11008	G	C
1	1:11012	0	11012	G	C
1	1:13110	0	13110	A	G
1	1:13116	0	13116	G	T
1	1:13118	0	13118	G	A
1	1:13273	0	13273	C	G
1	1:14464	0	14464	T	A
1	1:14599	0	14599	A	T
23	23:155258431	0	155258431	G	C
23	23:155258654	0	155258654	G	C
23	23:155259767	0	155259767	T	TA
23	23:155259783	0	155259783	GT	G
23	23:155259821	0	155259821	TG	T
23	23:155259882	0	155259882	A	AGGGTTAGGGTTAGGGGTTAG
23	23:155259986	0	155259986	A	AG
23	23:155260016	0	155260016	G	GTT
23	23:155260472	0	155260472	TG	T
23	23:155260478	0	155260478	TTA	T
  504  3024 12604 /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/mydataset.b37.rs.fam


In [8]:
%%bash
awk '{OFS="\t"; print "chr"$1,$4,$4+1,$2}' $Path/input/mydataset.b37.bim | sed 's/chr23/chrX/g' | sed 's/chr24/chrY/g' | sed 's/chr26/chrM/g' > $Path/output/UCSC_b37.bed
head $Path/UCSC_b37.bed

chr1	10177	10178	1:10177
chr1	10352	10353	1:10352
chr1	11008	11009	1:11008
chr1	11012	11013	1:11012
chr1	13110	13111	1:13110
chr1	13116	13117	1:13116
chr1	13118	13119	1:13118
chr1	13273	13274	1:13273
chr1	14464	14465	1:14464
chr1	14599	14600	1:14599


In [10]:
%%bash
# UCSC liftOver to get the same reference build. It creates two output files, hglft_genome.bed and unmapped.bed
# 1st parameter: bed file
# 2nd parameter: fixed path - do not change it
# 3rd parameter: path to output file
# 4th parameter: unmapped SNPs
$Path/liftOver $Path/output/UCSC_b37.bed $Path/ref_files/Phase3_v5/hg19ToHg18.over.chain.gz $Path/output/hglft_genome.bed $Path/output/unmapped.bed

Reading liftover chains
Mapping coordinates


In [11]:
%%bash
head $Path/hglft_genome.bed

chr1	177	178	1:10177
chr1	352	353	1:10352
chr1	871	872	1:11008
chr1	875	876	1:11012
chr1	2973	2974	1:13110
chr1	2979	2980	1:13116
chr1	2981	2982	1:13118
chr1	3136	3137	1:13273
chr1	4327	4328	1:14464
chr1	4462	4463	1:14599


In [22]:
%%bash
head /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt
sed 's/chr//g' /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt | awk 'BEGIN{OFS="\t";print "SNP","RS"};{OFS="\t"; if (NR>1) print $2":"$3,$4}' > /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.temp
head /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.temp
mv /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.temp > /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt

#bin	chrom	chromStart	name
1	chr1	16775073	rs72059099
1	chr1	16775613	rs71260122
1	chr1	16776311	rs72496645
2	chr1	100658226	rs71808286
14	chr1	45080556	rs71711725
26	chr1	149940941	rs71775793
30	chr1	177209325	rs71794609
33	chr1	202372482	rs72451924
35	chr1	220190850	rs72487224
SNP	RS
1:16775073	rs72059099
1:16775613	rs71260122
1:16776311	rs72496645
1:100658226	rs71808286
1:45080556	rs71711725
1:149940941	rs71775793
1:177209325	rs71794609
1:202372482	rs72451924
1:220190850	rs72487224


In [24]:
%%bash
head $Path/hglft_genome.bed

chr1	177	178	1:10177
chr1	352	353	1:10352
chr1	871	872	1:11008
chr1	875	876	1:11012
chr1	2973	2974	1:13110
chr1	2979	2980	1:13116
chr1	2981	2982	1:13118
chr1	3136	3137	1:13273
chr1	4327	4328	1:14464
chr1	4462	4463	1:14599


In [25]:
%%bash
awk '{OFS="\t"; print $4,$1}' $Path/hglft_genome.bed | sed 's/chrX/chr23/g' | sed 's/chrY/chr24/g' | sed 's/chrM/chr26/g'| sed 's/chr//g'  > $Path/update_chr.txt
awk '{OFS="\t"; print $4,$2}' $Path/hglft_genome.bed > $Path/update_bp.txt
awk '{OFS="\t"; print $4,$1":"$2}' $Path/hglft_genome.bed | sed 's/chr//g' > $Path/update_name.txt
awk '{OFS=""; print $1,":",$2}' $Path/hglft_genome.bed  | sed 's/chr//g' > $Path/UCSC_b36.txt
# fgrep -wf /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt > $Path/UCSC_b37_rs.txt


In [28]:
%%bash
head $Path/update_name.txt

1:10177	1:177
1:10352	1:352
1:11008	1:871
1:11012	1:875
1:13110	1:2973
1:13116	1:2979
1:13118	1:2981
1:13273	1:3136
1:14464	1:4327
1:14599	1:4462


In [26]:
%%bash
awk '{OFS="\t"; print $4}' $Path/unmapped.bed | sed '/^$/d' > $Path/rs_to_exclude
head $Path/rs_to_exclude
wc $Path/rs_to_exclude

1:2499283
1:5435388
1:5436036
1:5437120
1:5437257
1:5437512
1:5437521
1:5438738
1:5439068
1:5439069
  9891   9891 107260 /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/rs_to_exclude


In [30]:
%%bash
# Find plink logs in the Jupyter File view in this working directory
/mnt/Almacen6/Adapted/Tools/Plink/plink --bfile $Path/mydataset.b37 --exclude $Path/rs_to_exclude --update-chr $Path/update_chr.txt --allow-extra-chr --make-bed --zero-cms --out $Path/temp1
/mnt/Almacen6/Adapted/Tools/Plink/plink --bfile $Path/temp1 --update-map $Path/update_bp.txt --allow-extra-chr  --make-bed --out $Path/temp2
/mnt/Almacen6/Adapted/Tools/Plink/plink --bfile $Path/temp2 --update-name $Path/update_name.txt --allow-extra-chr --chr 1-26 --make-bed --out $Path/temp3

PLINK v1.90b6.17 64-bit (28 Apr 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp1.log.
Options in effect:
  --allow-extra-chr
  --bfile /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/mydataset.b37
  --exclude /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/rs_to_exclude
  --make-bed
  --out /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp1
  --update-chr /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/update_chr.txt
  --zero-cms

257718 MB RAM detected; reserving 128859 MB for main workspace.
Allocated 96644 MB successfully, after larger attempt(s) failed.
10061649 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
--exclude: 10051758 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and

/mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp1.hh ); many
commands treat these as missing.
/mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp2.hh ); many
commands treat these as missing.
treat these as missing.
/mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp3.hh ); many
commands treat these as missing.
treat these as missing.


In [33]:
%%bash
head $Path/temp3.bim
wc $Path/temp3.bim
head $Path/mydataset.b37.bim
wc $Path/mydataset.b37.bim

1	1:107	0	107	T	TA
1	1:177	0	177	AC	A
1	1:350	0	350	T	A
1	1:352	0	352	TA	T
1	1:352	0	352	GT	G
1	1:352	0	352	GT	G
1	1:352	0	352	GT	G
1	1:355	0	355	G	A
1	1:462	0	462	GT	G
1	1:871	0	871	G	C
 10048906  60293436 297555288 /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp3.bim
1	1:10177	0	10177	AC	A
1	1:10352	0	10352	TA	T
1	1:11008	0	11008	G	C
1	1:11012	0	11012	G	C
1	1:13110	0	13110	A	G
1	1:13116	0	13116	G	T
1	1:13118	0	13118	G	A
1	1:13273	0	13273	C	G
1	1:14464	0	14464	T	A
1	1:14599	0	14599	A	T
 10061649  60369894 298312818 /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/mydataset.b37.bim


In [35]:
%%bash
#  Remove SNP duplicates
sed 's/ /\t/g' $Path/temp3.bim  | awk '{print $2}' | sort | uniq -c| awk '{if($1>1) print $2}'> $Path/remove_duplicates.txt

In [37]:
%%bash
/mnt/Almacen6/Adapted/Tools/Plink/plink --bfile $Path/temp3 --exclude $Path/remove_duplicates.txt --make-bed --out $Path/temp4

PLINK v1.90b6.17 64-bit (28 Apr 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp4.log.
Options in effect:
  --bfile /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp3
  --exclude /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/remove_duplicates.txt
  --make-bed
  --out /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp4

257718 MB RAM detected; reserving 128859 MB for main workspace.
Allocated 96644 MB successfully, after larger attempt(s) failed.
10048906 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
--exclude: 10048790 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041

/mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp4.hh ); many
commands treat these as missing.
treat these as missing.


**Create the dataset in build 36 also in order to demonstrate in another notebook how to change builds.**

In [38]:
%%bash
/mnt/Almacen6/Adapted/Tools/Plink/plink --bfile $Path/temp4 --update-name /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt --make-bed --out /mnt/Almacen6/Adapted/jupyterAnalisis_others/Genomic_pipeline/dataset.b36
rm $Path/temp*

PLINK v1.90b6.17 64-bit (28 Apr 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/Almacen6/Adapted/jupyterAnalisis_others/Genomic_pipeline/dataset.b36.log.
Options in effect:
  --bfile /mnt/Almacen6/Adapted/protocol/1K_Genome/Phase3_v5_reduced/temp4
  --make-bed
  --out /mnt/Almacen6/Adapted/jupyterAnalisis_others/Genomic_pipeline/dataset.b36
  --update-name /mnt/Almacen6/Adapted/protocol/ref_files/UCSC.b36.rs.txt

257718 MB RAM detected; reserving 128859 MB for main workspace.
Allocated 96644 MB successfully, after larger attempt(s) failed.
10048790 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
--update-name: 0 values updated, 55617 variant IDs not present.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930

/mnt/Almacen6/Adapted/jupyterAnalisis_others/Genomic_pipeline/dataset.b36.hh );
many commands treat these as missing.
treat these as missing.
